In [1]:
from selenium import webdriver
import re
from pandas import DataFrame
import pickle
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

In [2]:
path = {'chrome':"C:/Users/preetham/Downloads/drivers/chromedriver.exe"
        ,'opera':"C:/Users/preetham/Downloads/drivers/operadriver.exe"}
url = "https://www.justdial.com/Mysore/Textile-Manufacturers/"
xpaths = {}
options = Options()
# options.add_argument("--headless") # Runs Chrome in headless mode.
# options.add_argument('--no-sandbox') # Bypass OS security model
# options.add_argument('--disable-gpu')  # applicable to windows os only
# options.add_argument('start-maximized') # 
# options.add_argument('disable-infobars')
# options.add_argument("--disable-extensions")
# capabilities = DesiredCapabilities.CHROME.copy()
# capabilities['acceptSslCerts'] = True 
# capabilities['acceptInsecureCerts'] = True

In [3]:
KEYWORD = 'Readymade Garment Manufacturers'
FILENAME = 'phbooktask2'

In [ ]:
# def scroll_to_bottom(driver):
    
#     SCROLL_PAUSE_TIME = 1
#     # Get scroll height
#     last_height = driver.execute_script("return document.body.scrollHeight")
 
#     while True:
#         # Scroll down to bottom
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
 
#         # Wait to load page
#         time.sleep(SCROLL_PAUSE_TIME)
 
#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height

In [ ]:
driver.get(url)

In [87]:
body = driver.find_element_by_css_selector('body')

In [88]:
body.send_keys(Keys.END)

In [89]:
con_main = driver.find_element_by_xpath('//*[@id="tab-5"]/ul')

In [90]:
soup = BeautifulSoup(con_main.get_attribute("innerHTML"),'lxml')
soup = soup.findAll('li')

In [7]:
import json

with open('cities.json') as f:
    cities = json.load(f)

In [8]:
for each in cities:
    if each["name"].lower() == "mysore":
        print(each)
        break

{'id': '104', 'name': 'Mysore', 'state': 'Karnataka'}


In [34]:
url = "https://www.justdial.com/"
city = input("city: ")
# keyword = input("keyword: ")
# keyword = '-'.join(keyword.split())
keyword = "textile-manufacturer"
url = url+city+'/'+keyword+'/'

city: bengaluru


In [95]:
total_count = 2842

In [4]:

def scraper(url,city,keyword,total_count):
#     flag = 0
    keyword = '-'.join(keyword.split())
#     if flag == 0:
#         url = url+city+'/'+keyword+'/'+"nct-10477448/page-23"
#         flag = 1
#     else:
#         url = url+city+'/'+keyword+'/'
    url = url+city+'/'+keyword+'/'

    marker = 0
    tt = pickle.load(open('num-mapper.pkl','rb'))
    while True:
        phbook = {}
        adrbook = {}
        tagbook = {} 
        if url!=None:
            print(url)
            if marker%2 == 0:
                driver = webdriver.Chrome(executable_path = path['chrome'],options = options)
            else:
                driver = webdriver.Opera(executable_path = path['opera'],options = options)
            driver.maximize_window()
            driver.implicitly_wait(3)
            marker+=1
            driver.get(url)
            body = driver.find_element_by_css_selector('body')
            time.sleep(1)
            body.send_keys(Keys.END)
        #     scroll_to_bottom(driver)
            time.sleep(10)
    #         con_main = driver.find_element_by_xpath('//*[@id="tab-5"]/ul')
            soup = BeautifulSoup(body.get_attribute("innerHTML"),'lxml')
            soup = soup.findAll('li')
            for i in range(len(soup)):
                try:
                    title = soup[i].h2.a['title']
                    address = soup[i].find("span",{"class":"mrehover dn"}).text
                    address = ' '.join(address.strip().split('\n'))
                    tags = soup[i].find("span",{"class":"margin0 addrinftxt"}).text
                    tags = ' '.join(tags.strip().split())
                    phno = ''
                    for each in soup[i].find("p",{"class":"contact-info"}).findAll('span')[1:]:
                        phno+=str(tt[re.findall('(icon-[\w]{2})',str(each))[0]])
    #                 try: 
    #                     phbook[title] += phno+','
    #                 except:
    #                     phbook[title] = ''
    #                     phbook[title] += phno+','
                    phbook[title] = phno
                    adrbook[title] = address
                    tagbook[title] = tags
                    total_count+=1
                    
                except:
                    pass
            nxt = driver.find_element_by_xpath('//a[@rel="next"]')
            url = nxt.get_attribute('href')
            driver.quit()
            df = DataFrame({'search-city':[city]*len(phbook.keys())
                            ,'search-key':[keyword]*len(phbook.keys())
                            ,'company':list(phbook.keys())
                            ,'phno':list(phbook.values())
                            ,"actual-address":list(adrbook.values())
                            ,"actual-tags":list(tagbook.values())})
            df.to_csv(FILENAME+'.csv',mode = 'a',header=False)
            print("==================running========================")
            print("total Count: ",total_count)
        else:
            break
    return total_count
    
    

In [5]:
total_count = 0
done = []

In [9]:

for each in cities[:5]:
    if each["name"] not in done:
        print(each["name"]+'--begining')
        total_count = scraper("https://www.justdial.com/",each["name"],"Readymade Garment Manufacturers",total_count)
        print("total count: ",total_count)
        print(each["name"], "is DONE!!!!!")
        done.append(each["name"])
        pickle.dump(done,open("done_cities.pkl","wb"))
        
        
        

Mumbai--begining
https://www.justdial.com/Mumbai/Readymade-Garment-Manufacturers/
==================running========================
total Count:  40
https://www.justdial.com/Mumbai/Readymade-Garment-Manufacturers/nct-10401890/page-2


SessionNotCreatedException: Message: session not created: This version of OperaDriver only supports Opera version 80
  (Driver info: operadriver=80.0.3987.100 (3f00c26d457663a424865bbef1179f72eec1b9fe-refs/branch-heads/3987@{#864}),platform=Windows NT 10.0.18362 x86_64)


In [83]:
pickle.dump(done,open("done.pickle","wb"))

In [ ]:
backup_url = https://www.justdial.com/Ahmedabad/Textile-Manufacturers/nct-10477448/page-40

In [16]:
if re.search('des',html):
    print("ka")

ka


In [86]:
DataFrame({'company':list(phbook.keys()),'phno':list(phbook.values())}).to_csv('demo.csv')

In [14]:
from pandas import read_csv

In [15]:
k = read_csv('phbook.csv')

In [17]:
k = k[['city','company','phno']]

In [19]:
k['search-key'] = None

In [20]:
k.head()

,city,company,phno,search-key
0,mysore,Morarjee Textiles Ltd,"+(91)-07103-281100,",None
1,mysore,V K T K Textiles,"+(91)-44-27224150,",None
2,mysore,Rajasthan Textile Mills,"+(91)-7432-222052,",None
3,mysore,Lakshmi Mills Company Ltd,"+(91)-422-2245461,",None
4,mysore,Parameswari Textiles,"+(91)-4324-221969,",None


In [84]:
url

'https://www.justdial.com/Bengaluru/Textile-Manufacturers/nct-10477448/page-2'

In [102]:
done.append("Jaipur")